In [3]:
import sys

sys.path.append("../")
from tiatoolbox.wsicore.wsireader import WSIReader
import numpy as np
import matplotlib.pyplot as plt
import os
from monkey.data.data_utils import (
    open_json_file,
    detection_to_annotation_store,
    extract_id,
)

# print(fetch_pretrained_weights("mapde-conic"))

Convert Ground Truth Annotations to AnnotationStore for visualization

In [5]:
annotations_folder = (
    "/home/u1910100/Downloads/Monkey/annotations/json"
)

save_dir = "/home/u1910100/Documents/Monkey/local_output/GT"

wsi_names = os.listdir(
    "/home/u1910100/Downloads/Monkey/images/pas-cpg"
)

for wsi_name in wsi_names:

    wsi_id = extract_id(wsi_name)
    inflammatory_json_name = f"{wsi_id}_inflammatory-cells.json"
    lymphocyte_json_name = f"{wsi_id}_lymphocytes.json"
    monocyte_json_name = f"{wsi_id}_monocytes.json"

    inflammatory_json_path = os.path.join(
        annotations_folder, inflammatory_json_name
    )
    lymphocytes_json_path = os.path.join(
        annotations_folder, lymphocyte_json_name
    )
    monocyte_json_path = os.path.join(
        annotations_folder, monocyte_json_name
    )

    inflammatory_annotations = open_json_file(inflammatory_json_path)
    inflammatory_points = inflammatory_annotations["points"]
    lymphocyte_annotations = open_json_file(lymphocytes_json_path)
    lymphocyte_points = lymphocyte_annotations["points"]
    monocyte_annotations = open_json_file(monocyte_json_path)
    monocyte_points = monocyte_annotations["points"]

    records = []
    for point in lymphocyte_points:
        records.append(
            {
                "x": point["point"][0],
                "y": point["point"][1],
                "type": "lymphocyte",
                "prob": 1.0,
            }
        )

    for point in monocyte_points:
        records.append(
            {
                "x": point["point"][0],
                "y": point["point"][1],
                "type": "monocyte",
                "prob": 1.0,
            }
        )

    store = detection_to_annotation_store(records, scale_factor=1.0)
    store_save_path = os.path.join(save_dir, f"{wsi_id}_true.db")
    store.dump(store_save_path)

Convert tif mask to png overlay

In [ ]:
mask_path = "/home/u1910100/Documents/Monkey/test/input/images/tissue-mask/A_P000001_mask.tif"
mask_reader = WSIReader.open(mask_path)
mask_thumbnail = mask_reader.slide_thumbnail()
binary_mask = mask_thumbnail * 255

plt.imshow(binary_mask)
plt.axis("off")
plt.show()

from PIL import Image

image = Image.fromarray(binary_mask)
image.save(
    "/home/u1910100/Documents/Monkey/local_output/A_P000001_PAS_CPG_mask.png"
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os


def load_nuclick_annotation(file_id, mask_dir):
    """
    Nuclick file format: 5 channel .np file
    channel 1-3: RGB image
    channel 4: instance map
    channel 5: class map (1:lymphocytes,2:monocytes)
    """
    mask_name = f"{file_id}.npy"
    mask_path = os.path.join(mask_dir, mask_name)
    mask = np.load(mask_path)
    mask = mask.astype(np.uint8)
    return mask[:, :, 4]


def class_mask_to_binary(class_mask: np.ndarray) -> np.ndarray:
    """Converts 2D cell class mask to binary mask
    Example:
        [1,0,0
         0,0,2
         0,0,1]
         ->
        [1,0,0
         0,0,1
         0,0,1]
    """
    binary_mask = np.zeros_like(class_mask)
    binary_mask[class_mask != 0] = 1
    return binary_mask


file_id = "D_P000019_9856_52192_10112_52448"
mask_dir = "./"

mask = load_nuclick_annotation(file_id, mask_dir)

print(mask.shape)
print(mask.dtype)

print(np.max(mask))
plt.imshow(mask)
plt.show()


binary_mask = class_mask_to_binary(mask)
plt.imshow(binary_mask)
plt.show()